# Completar panel de transacciones: cliente × producto × período

Este notebook genera un **panel completo de transacciones** entre clientes y productos para todos los períodos en los que ambos estuvieron activos.

---

## 🎯 Objetivo

Completar artificialmente las transacciones faltantes con `tn = 0` cuando:

- El `customer_id` estaba activo (tenía actividad de compra en algún producto),
- El `product_id` estaba activo (al menos algún cliente lo compró),
- Pero no se registró una transacción real en un `periodo` dado.

Esto permitirá obtener un panel denso y útil para entrenar modelos de forecasting que interpreten explícitamente los "ceros", es decir, la decisión activa de **no comprar**.

---

## 🔍 Reglas aplicadas

1. Un **cliente nace** en el primer período en que compró cualquier producto, y **muere** en su última compra.
2. Un **producto nace** en el primer período en que alguien lo compró, y **muere** en su última compra.
3. El producto y cliente **coexisten** en los períodos en que ambos están activos.
4. Si en ese período **no hubo transacción real**, se genera una artificial con `tn = 0`.


In [1]:

import pandas as pd
import numpy as np
from itertools import product


## 📥 Carga del archivo sell-in.txt

In [2]:

df = pd.read_csv("C:\\Developer\\Laboratorio_III\\data\\sell-in.txt", sep='\t')
df['periodo'] = df['periodo'].astype(str)
df.head

<bound method NDFrame.head of         periodo  customer_id  product_id  plan_precios_cuidados  \
0        201701        10234       20524                      0   
1        201701        10032       20524                      0   
2        201701        10217       20524                      0   
3        201701        10125       20524                      0   
4        201701        10012       20524                      0   
...         ...          ...         ...                    ...   
2945813  201912        10105       20853                      0   
2945814  201912        10092       20853                      0   
2945815  201912        10006       20853                      0   
2945816  201912        10018       20853                      0   
2945817  201912        10020       20853                      0   

         cust_request_qty  cust_request_tn       tn  
0                       2          0.05300  0.05300  
1                       1          0.13628  0.13628  
2  

## 📆 Determinar vida útil de clientes y productos

In [5]:

vida_clientes = df.groupby('customer_id')['periodo'].agg(['min', 'max']).rename(columns={'min': 'cliente_ini', 'max': 'cliente_fin'})
vida_productos = df.groupby('product_id')['periodo'].agg(['min', 'max']).rename(columns={'min': 'producto_ini', 'max': 'producto_fin'})
# 4. Lista de períodos únicos (ordenada)
periodos = sorted(df['periodo'].unique())


## 🧩 Generar universo de combinaciones válidas

In [6]:
# Obtener DataFrames individuales
clientes_df = vida_clientes.reset_index()
productos_df = vida_productos.reset_index()
periodos_df = pd.DataFrame({'periodo': periodos})

# Producto cartesiano: cliente × producto
cp_df = clientes_df.assign(key=1).merge(productos_df.assign(key=1), on='key').drop('key', axis=1)

# Calcular los rangos de actividad compartidos
cp_df['inicio_actividad'] = cp_df[['cliente_ini', 'producto_ini']].max(axis=1)
cp_df['fin_actividad'] = cp_df[['cliente_fin', 'producto_fin']].min(axis=1)

# Expandir con períodos (cliente-producto vivos)
cp_df = cp_df.merge(periodos_df, how='cross')
full = cp_df[
    (cp_df['periodo'] >= cp_df['inicio_actividad']) &
    (cp_df['periodo'] <= cp_df['fin_actividad'])
].copy()

# Marcar combinaciones válidas donde cliente y producto están vivos
full['flag_panel_valido'] = True



## 🔄 Fusionar con transacciones reales y completar con `tn = 0`

In [7]:

# Merge con datos reales
df_completo = full.merge(df[['customer_id', 'product_id', 'periodo', 'tn']], on=['customer_id', 'product_id', 'periodo'], how='left')

# Completar tn = 0 solo si el producto estaba activo en ese período
df_completo['tn'] = df_completo.apply(
    lambda row: 0 if pd.isna(row['tn']) and row['flag_panel_valido'] else row['tn'],
    axis=1
)

# Resultado final
df_completo = df_completo[['customer_id', 'product_id', 'periodo', 'tn']]


## 🧩 Agregar registros con tn = NaN cuando el producto ya existía pero el cliente aún no

### 🎯 Objetivo

Completar el panel de transacciones agregando registros con valor tn = NaN en aquellos casos en los que:

- El producto ya existía (estaba a la venta),
- Pero el cliente todavía no había comenzado su actividad,
- Y el producto seguía vigente cuando el cliente comenzó.

Esto permite reflejar en el panel que, si bien el cliente aún no estaba activo, el producto sí estaba disponible en el mercado. Es importante incluir estos puntos ya que constituyen períodos potenciales en los que aún no había posibilidad de compra, y por lo tanto, no se puede asumir cero como valor predeterminado.

In [11]:
# Preparar producto × cliente con fechas
pc_df = vida_productos.reset_index().assign(key=1).merge(
    vida_clientes.reset_index().assign(key=1), on='key'
).drop(columns='key')

# Filtrar combinaciones donde producto nació antes y cliente luego, pero aún dentro de la vida del producto
pc_df = pc_df[
    (pc_df['producto_ini'] < pc_df['cliente_ini']) &
    (pc_df['producto_fin'] >= pc_df['cliente_ini'])
].copy()

# Generar los períodos desde producto_ini hasta cliente_ini - 1
def generar_periodos(producto_ini, cliente_ini):
    producto_ini = int(producto_ini)
    cliente_ini = int(cliente_ini)
    periodos = []
    a1, m1 = divmod(producto_ini, 100)
    a2, m2 = divmod(cliente_ini, 100)

    while (a1 < a2) or (a1 == a2 and m1 < m2):
        periodos.append(a1 * 100 + m1)
        m1 += 1
        if m1 > 12:
            m1 = 1
            a1 += 1
    return periodos

# Expandir combinaciones a nivel período
registros_na = []
for _, row in pc_df.iterrows():
    for periodo in generar_periodos(row['producto_ini'], row['cliente_ini']):
        registros_na.append({
            'customer_id': row['customer_id'],
            'product_id': row['product_id'],
            'periodo': str(periodo),
            'tn': np.nan
        })

df_na = pd.DataFrame(registros_na)

# Unir al df_completo
df_completo = pd.concat([df_completo, df_na], ignore_index=True).drop_duplicates()


## 📈 Control de expansión del panel

Validamos cuántas combinaciones válidas fueron generadas en el panel expandido comparado con las transacciones originales:

- El panel completo debe contener **todas las combinaciones cliente-producto-período posibles** donde ambos estuvieron vivos.
- El resultado final debería expandir la cantidad de transacciones originales por un factor cercano a 5.8.

In [12]:
print(f"📊 Cantidad de registros generados: {len(df_completo):,}")
print(f"📦 Cantidad de registros originales: {len(df):,}")
print(f"🔁 Factor de expansión: {len(full) / len(df):.2f}x")

📊 Cantidad de registros generados: 17,022,744
📦 Cantidad de registros originales: 2,945,818
🔁 Factor de expansión: 5.28x


In [13]:
print("Registros únicos:", df_completo[['customer_id', 'product_id', 'periodo']].drop_duplicates().shape[0])
print("Registros totales:", df_completo.shape[0])

Registros únicos: 17022744
Registros totales: 17022744


## 💾 Guardar panel completo para análisis o modelado

In [14]:

df_completo.to_parquet("C:/Developer/Laboratorio_III/data/panel_completo_cliente_producto.parquet", index=False)
print("Archivo guardado: panel_completo_cliente_producto.parquet")


Archivo guardado: panel_completo_cliente_producto.parquet


## 🧪 Generación de dataset por producto y período

En esta sección generamos un nuevo dataset con el total de ventas y cantidad de clientes que compraron por `product_id` y `periodo`, enriquecido con características del producto y transformaciones de fecha útiles para modelos de forecasting.


In [15]:
# Agregamos toneladas totales y cantidad de clientes que compraron (tn > 0)
df_agg = df_completo.groupby(['product_id', 'periodo']).agg(
    tn_total=('tn', 'sum'),
    clientes_positivos=('tn', lambda x: (x > 0).sum())
).reset_index()


### 🧩 Enriquecimiento con atributos del producto

Importamos y unimos el maestro de productos (`tb_productos.txt`) para agregar categorías, marca y presentación (`sku_size`) a cada producto.


In [16]:
ruta_tb_productos = "C:/Developer/Laboratorio_III/data/tb_productos.txt"
df_productos = pd.read_csv(ruta_tb_productos, sep='\t')

# Merge con atributos del producto
columnas_uso = ['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
if 'descripcion' in df_productos.columns:
    columnas_uso.append('descripcion')

df_agg = df_agg.merge(df_productos[columnas_uso], on='product_id', how='left')


### 📆 Transformaciones de fecha

Convertimos la columna `periodo` al formato `datetime` para obtener nuevas variables como `mm-yyyy` y el trimestre (`quarter`).


In [17]:
df_agg['periodo'] = df_agg['periodo'].astype(str)
df_agg['fecha'] = pd.to_datetime(df_agg['periodo'], format='%Y%m')
df_agg['mm-yyyy'] = df_agg['fecha'].dt.strftime('%m-%Y')
df_agg['quarter'] = df_agg['fecha'].dt.to_period('Q').astype(str)


### 💾 Guardado del dataset resultante

Exportamos el dataset enriquecido en formato `.parquet` para facilitar su uso posterior en entrenamiento de modelos de forecasting.


In [18]:
df_agg.to_parquet("C:/Developer/Laboratorio_III/data/dataset_product_periodo.parquet", index=False)
print("✅ Dataset guardado como 'dataset_product_periodo.parquet'")


✅ Dataset guardado como 'dataset_product_periodo.parquet'
